## 思いつきで単発作成するFeature

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime
import time
from tqdm import tqdm
import sys
import glob

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
# regist_termをもたせる
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)
train_test.set_index(key, inplace=True)
base = utils.read_df_pkl('../input/base_term*0*').set_index(key)[['hist_regist_term', 'new_regist_term']]
train_test = train_test.join(base)

df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  1.54it/s]


SystemExit: 

In [12]:
key_list = df_new[key].drop_duplicates().values
l = train_test.loc[key_list].shape[0]
len(train_test) - l

35539

In [4]:
#========================================================================
# 登録より前に購買が発生しているユーザーかどうかわかるdiff term 
prefix = '555_elo'
base = utils.read_df_pkl('../input/base_term*0*')
base[['hist_regist_term','hist_personal_term']].groupby(['hist_regist_term','hist_personal_term']).size()
base["diff_regist_term_hist_term"] = base['hist_regist_term'] - base['hist_personal_term']
utils.to_pkl_gzip(obj=base["diff_regist_term_hist_term"], path=f'../features/1_first_valid/{prefix}_diff_regist_term_hist_term')
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 18.93it/s]


In [7]:
#========================================================================
# CLV
df_hist = df_hist[['card_id','purchase_date','purchase_amount']]

## Time
from datetime import datetime

z = df_hist.groupby('card_id')['purchase_date'].max().reset_index()
q = df_hist.groupby('card_id')['purchase_date'].min().reset_index()

z.columns = ['card_id', 'Max']
q.columns = ['card_id', 'Min']

## Extracting current timestamp
now = datetime.now()
curr_date = now.strftime("%m-%d-%Y, %H:%M:%S")
curr_date = pd.to_datetime(curr_date)

rec = pd.merge(z,q, how = 'left',on = 'card_id')
rec['Min'] = pd.to_datetime(rec['Min'])
rec['Max'] = pd.to_datetime(rec['Max'])

## Time value 
rec['Recency'] = (curr_date - rec['Max']).astype('timedelta64[D]') ## current date - most recent date

## Recency value
rec['Time'] = (rec['Max'] - rec['Min']).astype('timedelta64[D]') ## Age of customer, MAX - MIN

rec = rec[['card_id','Time','Recency']]

## Frequency
freq = df_hist.groupby('card_id').size().reset_index()
freq.columns = ['card_id', 'Frequency']
## Monitary
mon = df_hist.groupby('card_id')['purchase_amount'].sum().reset_index()
mon.columns = ['card_id', 'Monitary']

final = pd.merge(freq,mon,how = 'left', on = 'card_id')
final = pd.merge(final,rec,how = 'left', on = 'card_id')

final['historic_CLV'] = final['Frequency'] * final['Monitary'] 
final['AOV'] = final['Monitary']/final['Frequency'] ## AOV - Average order value (i.e) total_purchase_amt/total_trans
final['Predictive_CLV'] = final['Time']*final['AOV']*final['Monitary']*final['Recency'] 

for col in ['AOV', 'Predictive_CLV']:
    feature = final[col].fillna(-1).values.astype('float32')
    print(feature.shape)
    utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_{col}')
#========================================================================

,card_id,Time,Recency
0,C_ID_00007093c1,377.0,343.0
1,C_ID_0001238066,151.0,342.0
2,C_ID_0001506ef0,398.0,352.0
3,C_ID_0001793786,283.0,461.0
4,C_ID_000183fdda,202.0,344.0


In [9]:
#========================================================================
# feature_setをロード
win_path = f'../features/4_winner/*.gz'
# win_path = f'../features/0_exp/*.gz'
win_path_list = glob.glob(win_path)
base = utils.read_df_pkl('../input/base_term*0*')
feature_list = utils.parallel_load_data(path_list=win_path_list)

df_feat = pd.concat(feature_list, axis=1)
df_feat = pd.concat([base, df_feat], axis=1)
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 17.56it/s]


In [11]:
# ========================================================================
# Featureをnew_regist_term別にliftにする
# train_testの粒度でliftを作る
feat_cols = sorted([col for col in df_feat.columns if col.count('@')])
select_cols = feat_cols
lift_level = 'hist_regist_term'
prefix = f'556_lif_{lift_level}'
prefix = f'557_lif_{lift_level}'
prefix = f'559_lif_{lift_level}'

level_list = np.unique(df_feat[lift_level].values)
df_list = []

for col in tqdm(select_cols):
    tmp_list = []
    for level in level_list:
        tmp = df_feat[df_feat[lift_level]==level][[key, col]]
        base_avg = tmp[col].mean()
        tmp[col] = tmp[col] / base_avg
        tmp.rename(columns={col:f'lift_{col}'}, inplace=True)
        tmp_list.append(tmp)
        
    df_level = pd.concat(tmp_list, axis=0)
    print(df_level.shape)
#     df_level[lift_level] = level
    df_level.set_index(key, inplace=True)
    df_list.append(df_level)
        
df_lift = pd.concat(df_list, axis=1)
df_lift = df_lift.reset_index().set_index(key)
print(df_lift.shape)

df_lift = base[[key, target]].set_index(key).join(df_lift)

for col in df_lift.columns:
    if not(col.count('lift')):continue
    feature = df_lift[col].fillna(-1).values.astype('float32')
    print(feature.shape)
    utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_{col}')
#========================================================================

  1%|          | 1/112 [00:01<03:01,  1.63s/it]

(325540, 2)


  2%|▏         | 2/112 [00:02<02:33,  1.40s/it]

(325540, 2)


  3%|▎         | 3/112 [00:03<02:13,  1.23s/it]

...

 99%|█████████▉| 111/112 [01:33<00:00,  1.19it/s]

(325540, 2)


100%|██████████| 112/112 [01:34<00:00,  1.19it/s]

(325540, 2)


(325540, 112)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(32554

In [ ]:
# ========================================================================
# Featureをhist_regist_termで割り算する
# train_testの粒度でliftを作る
feat_cols = sorted([col for col in df_feat.columns if col.count('@') and not(col.count('lift'))])
select_cols = feat_cols
div_level = 'hist_regist_term'
prefix = f'558_div_{div_level}'

level_list = np.unique(df_feat[lift_level].values)
df_list = []

for col in tqdm(select_cols):
    df_feat[f'div_{col}'] = df_feat[col] / df_feat[div_level]
print(df_feat.shape)
        
for col in df_feat.columns:
    if not(col.count('div_')):continue
#     feature = df_feat[col].fillna(-1).values.astype('float32')
    feature = df_feat[col].values.astype('float32')
    print(feature.shape)
    utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_{col}')
#========================================================================